In [1]:
import torch
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 
import os
from shutil import copy

In [8]:
main_path = r"/home/adrian/Documents/academia/UAB/year_3/vision_learning/project1/project1_repo/deep_learning_method/archive"

In [9]:
import xml.etree.ElementTree as ET
annotation_df = pd.DataFrame(columns = ['FileName', 'xmin','ymin','xmax','ymax',
                                        'width','height'])

path_xml = main_path + "/annotations"
for filename in os.listdir(path_xml):
    annotation_list = []
    tree = ET.parse(os.path.join(path_xml,filename))
    root = tree.getroot()
    annotation_list.append(root.find('filename').text)
    #print(root.find('filename').text)
    for cord in root.find('object').find('bndbox'):
        annotation_list.append(int(cord.text))
    annotation_list.append(int(root.find('size').find('width').text))
    annotation_list.append(int(root.find('size').find('height').text))
    annotation_df.loc[len(annotation_df)] = annotation_list
    
annotation_df['center_x'] = (annotation_df['xmax'] + annotation_df['xmin'])/(2*annotation_df['width'])
annotation_df['center_y'] = (annotation_df['ymax'] + annotation_df['ymin'])/(2*annotation_df['height'])

annotation_df['bb_width'] = (annotation_df['xmax'] - annotation_df['xmin'])/annotation_df['width']
annotation_df['bb_height'] = (annotation_df['ymax'] - annotation_df['ymin'])/annotation_df['height']

annotation_df

,FileName,xmin,ymin,xmax,ymax,width,height,center_x,center_y,bb_width,bb_height
0,Cars38.png,168,196,240,231,386,400,0.528497,0.533750,0.186528,0.087500
1,Cars134.png,76,151,133,174,400,267,0.261250,0.608614,0.142500,0.086142
2,Cars153.png,146,118,311,170,471,270,0.485138,0.533333,0.350318,0.192593
3,Cars161.png,312,283,406,311,507,388,0.708087,0.765464,0.185404,0.072165
4,Cars394.png,291,167,410,190,555,416,0.631532,0.429087,0.214414,0.055288
...,...,...,...,...,...,...,...,...,...,...,...
428,Cars404.png,162,208,235,244,370,400,0.536486,0.565000,0.197297,0.090000
429,Cars70.png,179,115,213,133,400,226,0.490000,0.548673,0.085000,0.079646
430,Cars29.png,143,171,211,208,400,299,0.442500,0.633779,0.170000,0.123746
431,Cars395.png,241,182,288,205,400,301,0.661250,0.642857,0.117500,0.076412


In [10]:
df_train, df_test_val = train_test_split(annotation_df, test_size=0.25)
df_test, df_val = train_test_split(df_test_val, test_size=0.50)
[len(df_train), len(df_test), len(df_val)]

[324, 54, 55]

In [13]:
train_folder = './yolo_data/'
values = df_train[['FileName','center_x','center_y','bb_width','bb_height']].values
for fname, x, y, w, h in values:
    label_txt = f'0 {x} {y} {w} {h}'
    copy(os.path.join(main_path + "/images",fname),
         os.path.join(train_folder + "/images/train",fname))
    fname_ = fname.split('.')[0]
    label_path = os.path.join(train_folder + '/labels/train',fname_+'.txt')
    with open(label_path, mode='w') as f:
        f.write(label_txt)
        f.close()

In [14]:
test_folder = './yolo_data'
values = df_test[['FileName','center_x','center_y','bb_width','bb_height']].values
for fname, x, y, w, h in values:
    label_txt = f'0_data {x} {y} {w} {h}'
    copy(os.path.join(main_path + "/images",fname),
         os.path.join(train_folder + "/images/test",fname))
    fname_ = fname.split('.')[0]
    label_path = os.path.join(test_folder + '/labels/test',fname_+'.txt')
    with open(label_path, mode='w') as f:
        f.write(label_txt)
        f.close()

In [15]:
test_folder = './yolo_data'
values = df_val[['FileName','center_x','center_y','bb_width','bb_height']].values
for fname, x, y, w, h in values:
    label_txt = f'0 {x} {y} {w} {h}'
    copy(os.path.join(main_path + "/images",fname),
         os.path.join(test_folder + "/images/val",fname))
    fname_ = fname.split('.')[0]
    label_path = os.path.join(test_folder + '/labels/val',fname_+'.txt')
    with open(label_path, mode='w') as f:
        f.write(label_txt)
        f.close()